In [1]:
import cv2
import numpy as  np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,concatenate,AveragePooling2D,Rescaling
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Rescaling
from keras.preprocessing.image import img_to_array

In [2]:
def crop_minAreaRect(rect, box, gray):
    mult = 1 
    W = rect[1][0]
    H = rect[1][1]

    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)

    rotated = False
    angle = rect[2]

    if angle < -45:
        angle+=90
        rotated = True

    center = (int((x1+x2)/2), int((y1+y2)/2))
    size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

    M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

    cropped = cv2.getRectSubPix(gray, size, center)    
    cropped = cv2.warpAffine(cropped, M, size)

    croppedW = W if not rotated else H 
    croppedH = H if not rotated else W

    croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))
    
    return croppedRotated

In [3]:
def inception_layer(prev_layer, filters_1_1x1=16, filters_2_1x1=24, filters_2_3x3=32, filters_3_1x1=4, filters_3_5x5=8, filters_4_1x1=8, name=None):
  conv1 = Conv2D(filters=filters_1_1x1, kernel_size=(1,1), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(prev_layer)

  conv2 = Conv2D(filters=filters_2_1x1, kernel_size=(1,1), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(prev_layer)
  conv2 = Conv2D(filters=filters_2_3x3, kernel_size=(3,3), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(conv2)

  conv3 = Conv2D(filters=filters_3_1x1, kernel_size=(1,1), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(prev_layer)
  conv3 = Conv2D(filters=filters_3_5x5, kernel_size=(5,5), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(conv3)

  conv4 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(prev_layer)
  conv4 = Conv2D(filters=filters_4_1x1, kernel_size=(1,1), padding='same', activation='relu', bias_initializer=tf.keras.initializers.Constant(value=0.2))(conv4)

  return concatenate([conv1, conv2, conv3, conv4], axis=3, name=name)

In [4]:
input_shape = (64, 64, 3)


inputs = tf.keras.Input(shape=input_shape)
x=  Rescaling(1.0 / 255)(inputs)
x = Conv2D(filters=16, kernel_size=(7,7), strides=(2,2), padding='same', activation='relu', name='conv1_7x7/2', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x)

x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='maxpool2_3x3/2')(x)
x = Conv2D(filters=16, kernel_size=(1,1), padding='same', activation='relu', name='conv2_1x1/1', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x)
x = Conv2D(filters=48, kernel_size=(3,3), padding='same', activation='relu', name='conv2_3x3/1', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='maxpool3_3x3/2')(x)

x = inception_layer(x, filters_1_1x1=16, filters_2_1x1=24, filters_2_3x3=32, filters_3_1x1=4, filters_3_5x5=8, filters_4_1x1=8, name='inception3_1')
x = inception_layer(x, filters_1_1x1=32, filters_2_1x1=32, filters_2_3x3=48, filters_3_1x1=8, filters_3_5x5=24, filters_4_1x1=16, name='inception3_2')
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='maxpool4_3x3/2')(x)

x = inception_layer(x, filters_1_1x1=48, filters_2_1x1=24, filters_2_3x3=52, filters_3_1x1=4, filters_3_5x5=12, filters_4_1x1=16, name='inception4_1')
## first output
x1 = AveragePooling2D(pool_size=(5, 5), strides=(3,3), padding='same', name='out1_avgpool_5x5/3')(x)
x1 = Conv2D(filters=32, kernel_size=(1,1), padding='same', activation='relu', name='out1_conv_1x1/1', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x1)
x1 = Flatten()(x1)
x1 = Dense(64, activation='relu', name='out1_dense1')(x1)
x1 = Dropout(0.7, name='out1_dropout')(x1)
x1 = Dense(1, activation='sigmoid', name='out1')(x1)

x = inception_layer(x, filters_1_1x1=40, filters_2_1x1=28, filters_2_3x3=56, filters_3_1x1=6, filters_3_5x5=16, filters_4_1x1=16, name='inception4_2')
x = inception_layer(x, filters_1_1x1=32, filters_2_1x1=32, filters_2_3x3=64, filters_3_1x1=6, filters_3_5x5=16, filters_4_1x1=16, name='inception4_3')
x = inception_layer(x, filters_1_1x1=28, filters_2_1x1=36, filters_2_3x3=72, filters_3_1x1=8, filters_3_5x5=16, filters_4_1x1=16, name='inception4_4')
## second output
x2 = AveragePooling2D(pool_size=(5, 5), strides=(3,3), padding='same', name='out2_avgpool_5x5/3')(x)
x2 = Conv2D(filters=32, kernel_size=(1,1), padding='same', activation='relu', name='out2_conv_1x1/1', bias_initializer=tf.keras.initializers.Constant(value=0.2))(x2)
x2 = Flatten()(x2)
x2 = Dense(64, activation='relu', name='out2_dense1')(x2)
x2 = Dropout(0.7, name='out2_dropout')(x2)
x2 = Dense(1, activation='sigmoid', name='out2')(x2)

x = inception_layer(x, filters_1_1x1=64, filters_2_1x1=40, filters_2_3x3=80, filters_3_1x1=8, filters_3_5x5=32, filters_4_1x1=32, name='inception4_5')
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='maxpool5_3x3/2')(x)

x = inception_layer(x, filters_1_1x1=64, filters_2_1x1=40, filters_2_3x3=80, filters_3_1x1=8, filters_3_5x5=32, filters_4_1x1=32, name='inception5_1')
x = inception_layer(x, filters_1_1x1=96, filters_2_1x1=48, filters_2_3x3=96, filters_3_1x1=12, filters_3_5x5=32, filters_4_1x1=32, name='inception5_2')
## third output
x = AveragePooling2D(pool_size=(7, 7), strides=(1,1), padding='same', name='out3_avgpool_7x7/1')(x)
x = Flatten()(x)
x = Dropout(0.7, name='out3_dropout')(x)
x = Dense(1, activation='sigmoid', name='out3')(x)

model = tf.keras.Model(inputs=inputs, outputs=[x, x1, x2], name='Inception-v1')
model.summary()


Model: "Inception-v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 64, 64, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_7x7/2 (Conv2D)           (None, 32, 32, 16)   2368        ['rescaling[0][0]']              
                                                                                                  
 maxpool2_3x3/2 (MaxPooling2D)  (None, 16, 16, 16)   0           ['conv1_7x7/2[0][0]']            
                                                                                       

In [5]:
model = tf.keras.models.load_model("model_inception.h5")

In [6]:
img_crop=cv2.imread("0.jpg",1)
img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
img_crop = img_to_array(img_crop)
img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
Y_box_pred = model.predict(img_crop)[2]
print(Y_box_pred)

[[1.]]


In [7]:
video_path='test2.mp4'
cap=cv2.VideoCapture(video_path)
backSub=cv2.createBackgroundSubtractorMOG2()
if(cap.isOpened()==False):
    print("Khong mo duoc video")
while (cap.isOpened()):
    ret, frame=cap.read()
    if ret==True:
        #Thuat toan tru nen
        
        fgMask=backSub.apply(frame)
        fgMask=cv2.cvtColor(fgMask, 0)
        #Dung bo loc ma tran 5*5
        kernel = np.ones((7,7), np.uint8)
        #Lay gia tri pixel nho nhat cua kernel lam manh net
        fgMask=cv2.erode(fgMask,kernel,iterations=1)
        #Lay gia tri pixel lon nhat cua kernel lam lien net
        fgMask=cv2.dilate(fgMask,kernel,iterations=1)
        
        #tao them su lien net bang ham open
        fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel,iterations=1)
        #lay nguong 120
        _,fgMask = cv2.threshold(fgMask,120,255,cv2.THRESH_BINARY)
        #phat hien canh bang candy
        fgMask=cv2.Canny(fgMask, 20, 200)
        contours,_ = cv2.findContours(fgMask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i in range(len(contours)):
            rect = cv2.minAreaRect(contours[i])
            if (rect[1][0]*rect[1][1]) < 1000:
                continue
            
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            img_crop = crop_minAreaRect(rect, box, frame)       
            img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
            img_crop = img_to_array(img_crop)
            img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
            cv2.drawContours(frame,[box],0,(0,255,0),2)
        
        
            Y_box_pred = model.predict(img_crop)[2]
            if Y_box_pred <= 0.5:
                cv2.putText(frame, "Person", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            if Y_box_pred > 0.5:
                cv2.putText(frame, "Non", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        cv2.imshow('Frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
cv2.destroyAllWindows()    
            